In [1]:
import cv2
import numpy as np
import pywt
from pathlib import Path

In [2]:
# ---------- I/O ----------
# Path to host (cover) image – use any 8-bit grayscale or color file
img_path = Path(r"images\chihuahua.webp")
img      = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
if img is None: 
    raise FileNotFoundError(img_path)

cv2.imshow("Host Image", img)
cv2.waitKey(0)

-1

In [3]:
# ---------- 4-Level Forward DWT ----------
wavelet = "haar"                     # or 'db2', 'bior4.4', etc.
level   = 4
coeffs  = pywt.wavedec2(img, wavelet=wavelet, level=level)

# coeffs structure:  [cA4, (cH4,cV4,cD4), (cH3,cV3,cD3), …, (cH1,cV1,cD1)]
cA4, detail_coeffs = coeffs[0], coeffs[1:]

print("LL4 (approximation) shape :", cA4.shape)
for i, (cH, cV, cD) in enumerate(detail_coeffs[::-1], start=1):
    print(f"Level {i} —  LH{5-i}:{cH.shape}, HL{5-i}:{cV.shape}, HH{5-i}:{cD.shape}")

LL4 (approximation) shape : (68, 68)
Level 1 —  LH4:(540, 540), HL4:(540, 540), HH4:(540, 540)
Level 2 —  LH3:(270, 270), HL3:(270, 270), HH3:(270, 270)
Level 3 —  LH2:(135, 135), HL2:(135, 135), HH2:(135, 135)
Level 4 —  LH1:(68, 68), HL1:(68, 68), HH1:(68, 68)


In [4]:
# ---------- ⬇️ WATERMARK EMBEDDING POINT -------------------
# Example: embed in HL4 band (cV at deepest level)
# (Below just adds tiny noise to illustrate the spot; replace with your own logic.)

wm_strength = 4      # small integer to keep imperceptible
rng         = np.random.default_rng(42)

# Modify *one* sub-band (e.g., HL4) for robustness
cH4, cV4, cD4 = detail_coeffs[-1]           # deepest level bands
embed_bits  = rng.integers(-wm_strength, wm_strength + 1, size=cV4.shape, dtype=np.int16)
cV4_watermarked = cV4 + embed_bits          # embed by simple addition

# Re-assemble the coefficient list with the modified band
detail_coeffs[-1] = (cH4, cV4_watermarked, cD4)
coeffs_watermarked = [cA4] + detail_coeffs

In [5]:
# ---------- Inverse DWT (reconstruction) ----------
watermarked_img = pywt.waverec2(coeffs_watermarked, wavelet).astype(np.uint8)

In [6]:
# ---------- Save / inspect ----------
# cv2.imwrite("watermarked.png", watermarked_img)
# print("Watermarked image saved → watermarked.png")

cv2.imshow("Watermarked Image", watermarked_img)
cv2.waitKey(0)

-1

In [26]:
# PSNR (Peak-Signal-to-Noise Ratio)
def psnr(img1, img2, max_val=255.0):
    mse = np.mean((img1.astype(np.float32) - img2.astype(np.float32))**2)
    return 20 * np.log10(max_val) - 10 * np.log10(mse)

# Bit-Error-Rate
def ber(original, recovered):
    diff = np.sum(original != recovered)
    return diff / original.size

In [27]:
psnr_val = psnr(img, watermarked_img)
print(f"PSNR host vs. watermarked: {psnr_val:.2f} dB")

PSNR host vs. watermarked: 45.63 dB


In [28]:
# ------------------------------------------------------------
# 3️⃣  Extract watermark & BER
# ------------------------------------------------------------
# 💡  In a real attack you would start from a **received** image (possibly compressed).
#      Here we use the freshly generated one just to show the pipeline.
wm_bits      = rng.choice([-1, 1], size=cV4.shape, replace=True)   # original watermark
coeffs_rcv       = pywt.wavedec2(watermarked_img, wavelet=wavelet, level=level)
cA4_rcv, det_rcv = coeffs_rcv[0], coeffs_rcv[1:]
cH4_rcv, cV4_rcv, cD4_rcv = det_rcv[-1]                       # HL4 band again

# simple extraction: sign(cV4_rcv - original_cV4)  ⇒ ±1
extracted_bits = np.sign(cV4_rcv - cV4).astype(int)
# replace zeros (should be rare) with +1
extracted_bits[extracted_bits == 0] = 1

ber_val = ber(wm_bits, extracted_bits)
print(f"BER (original ↔ extracted watermark): {ber_val:.4f}")

BER (original ↔ extracted watermark): 0.5007


In [39]:
# ------------------------------------------------------------
# Robust watermarking by DWT-coefficient fusion
# ------------------------------------------------------------
# ⬇️  install once per Colab / venv
# !pip install opencv-python pywavelets numpy

import cv2, pywt, numpy as np
from pathlib import Path

# ========== 1. Load host & watermark ==========
host_path      = Path(r"images\chihuahua.webp")        # any 8-bit gray image, e.g. 512×512
watermark_path = Path(r"images\watermark.jpg")          # a *smaller* gray logo, e.g. 64×64
host_img       = cv2.imread(str(host_path), cv2.IMREAD_GRAYSCALE)
wm_img         = cv2.imread(str(watermark_path), cv2.IMREAD_GRAYSCALE)
if host_img is None or wm_img is None:
    raise FileNotFoundError("Check host / wm paths")

# optional: make watermark square & power-of-two for nicer shapes
host_img = cv2.resize(host_img, (512, 512), interpolation=cv2.INTER_AREA)
wm_img = cv2.resize(wm_img, (64, 64), interpolation=cv2.INTER_AREA)

# ========== 2. Forward DWTs ==========
wavelet, host_level = "haar", 4
wm_level            = 1

coeffs_host = pywt.wavedec2(host_img, wavelet=wavelet, level=host_level)
cA4, detail_host   = coeffs_host[0], coeffs_host[1:]        # host coeff stack

coeffs_wm  = pywt.wavedec2(wm_img,  wavelet=wavelet, level=wm_level)
cA_wm, (cH_wm, cV_wm, cD_wm) = coeffs_wm                   # watermark LL₁ + details

# ========== 3. Coefficient-fusion rule ==========
# We embed watermark’s LL₁ into host’s HL₄ (cV4) after resizing if shapes differ.
cH4, cV4, cD4           = detail_host[0]                   # HL₄, etc.
wm_resized              = cv2.resize(cA_wm, cV4.shape[::-1], cv2.INTER_LINEAR)
alpha                   = 0.12                               # embedding strength
cV4_wm                  = cV4 + alpha * wm_resized          # fusion

# put back the modified sub-band
detail_host[0]         = (cH4, cV4_wm, cD4)
coeffs_fused            = [cA4] + detail_host

# ========== 4. Inverse DWT ⇒ watermarked image ==========
watermarked_img = (
    pywt.waverec2(coeffs_fused, wavelet)
    .clip(0, 255)
    .astype(np.uint8)
)
cv2.imwrite("watermarked.png", watermarked_img)
print("✅ Watermarked image saved → watermarked.png")

# ========== 5. PSNR (cover ↔ watermarked) ==========
def psnr(a, b, max_val=255.0):
    mse = np.mean((a.astype(np.float32) - b.astype(np.float32)) ** 2)
    return 20 * np.log10(max_val) - 10 * np.log10(mse)

print(f"PSNR: {psnr(host_img, watermarked_img):.2f} dB")

# ========== 6. Extract watermark & BER ==========
# (In practice, start from an attacked/received image.)
coeffs_rcv            = pywt.wavedec2(watermarked_img, wavelet=wavelet, level=host_level)
cA4_rcv, det_rcv      = coeffs_rcv[0], coeffs_rcv[1:]
cH4_rcv, cV4_rcv, _   = det_rcv[0]

extracted_LL1         = (cV4_rcv - cV4) / alpha            # reverse the fusion
extracted_LL1         = np.round(extracted_LL1).astype(np.int16)

# quick binary decision (assuming original watermark is 0/255)
orig_bits      = (cA_wm > 127).astype(np.uint8)
recv_bits      = (extracted_LL1 > 127).astype(np.uint8)
ber            = np.mean(orig_bits != recv_bits)
print(f"BER:  {ber:.4f}")

# Optionally rebuild the watermark image for visual inspection
coeffs_wm_rcv  = [extracted_LL1, (np.zeros_like(cH_wm),
                                  np.zeros_like(cV_wm),
                                  np.zeros_like(cD_wm))]
wm_rcv_img     = pywt.waverec2(coeffs_wm_rcv, wavelet).clip(0,255).astype(np.uint8)
cv2.imwrite("wm_extracted.png", wm_rcv_img)
print("✅ Extracted watermark saved → wm_extracted.png")


✅ Watermarked image saved → watermarked.png
PSNR: 39.11 dB
BER:  0.0156
✅ Extracted watermark saved → wm_extracted.png
